In [20]:
import pandas as pd
import numpy as np
from itertools import chain
from collections import Counter, defaultdict
import pickle
import re
from tqdm.notebook import tqdm, trange

In [2]:
from dataloader import *

Data from "data/ao3_ver1_full.p" loaded. Document Size: 84866. System Memory: 4040.70mb


In [3]:
imdb = pd.read_csv('data/imdb.tsv',  sep='\t', header=0)
rating = pd.read_csv('data/ratings.tsv', sep='\t', header=0)

C:\Users\Ariel\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3155: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
imdb = imdb.merge(rating, how='left', on=['tconst'])
imdb = imdb[imdb['numVotes'].notna()]
imdb = imdb[~imdb['titleType'].isin(['tvEpisode', 'tvSpecial', 'video', 'short'])]
imdb = imdb.reset_index(drop=True)

In [5]:
imdb

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0000502,movie,Bohemios,Bohemios,0,1905,\N,100,\N,4.5,8.0
1,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,\N,70,"Action,Adventure,Biography",6.1,632.0
2,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,\N,90,Drama,6.0,5.0
3,tt0000615,movie,Robbery Under Arms,Robbery Under Arms,0,1907,\N,\N,Drama,4.5,14.0
4,tt0000630,movie,Hamlet,Amleto,0,1908,\N,\N,Drama,3.8,12.0
...,...,...,...,...,...,...,...,...,...,...,...
412029,tt9916362,movie,Coven,Akelarre,0,2020,\N,92,"Adventure,Drama,History",6.3,3218.0
412030,tt9916380,tvSeries,Meie aasta Aafrikas,Meie aasta Aafrikas,0,2019,\N,43,"Adventure,Comedy,Family",9.0,104.0
412031,tt9916428,movie,The Secret of China,Hong xing zhao yao Zhong guo,0,2019,\N,\N,"Adventure,History,War",3.8,12.0
412032,tt9916460,tvMovie,Pink Taxi,Pink Taxi,0,2019,\N,\N,Comedy,9.3,15.0


In [7]:
titles = imdb['primaryTitle'].tolist()

In [14]:
np.unique(imdb['titleType'])

array(['movie', 'tvMiniSeries', 'tvMovie', 'tvSeries', 'tvShort',
       'videoGame'], dtype=object)

In [17]:
def search_for_closest(query):
    ids = []
    for idx, t in enumerate(titles):
        if query in str(t):
            ids.append(idx)
    ranked = imdb.iloc[ids].sort_values(by='numVotes', ascending=False)
    if len(ranked) > 0:
        return ranked.iloc[0].to_dict()
    return {}

In [18]:
fandom_count = Counter(chain.from_iterable(fandoms))
fandom_total = len(fandom_count)

In [19]:
def parse_fandom(fandom):
    f = fandom.split(' | ')[-1]
    match_obj = re.match(r'([^\(\)\-]+)($|\(.+\)|- All Media Types)?', f)
    if not match_obj:
        return f
    else:
        return match_obj[1].strip(' '), match_obj[2].replace(')', '').replace('(', '') if match_obj[2] else ''

In [21]:
data = []
for fandom, count in tqdm(sorted(fandom_count.items(), key=lambda x:-x[1])):
    parsed = parse_fandom(fandom)
    res = search_for_closest(parsed[0])
    if res:
        res['numVotes'] = int(res['numVotes'])
        res['genres'] = res['genres'].split(',')
        if 'tv' in res['titleType']:
            res['titleType'] = 'tv'
        elif 'video' in res['titleType']:
            res['titleType'] = 'video_game'
    
    entry = {
        'ao3_name': fandom,
        'count': count,
        'percentage': round(count / fandom_total, 4),
        'imdb_name': res.get('primaryTitle', ''),
        'media_type': res.get('titleType', ''),
        'genres': res.get('genres', ''),
        'votes': res.get('numVotes', ''),
        'start_year': res.get('startYear', ''),
    }
    data.append(entry)

  0%|          | 0/7015 [00:00<?, ?it/s]

In [22]:
fandom_df = pd.DataFrame(data)

In [24]:
pickle.dump(fandom_df, open('data/fandom.p', 'wb'))